In [4]:
import torch
import torch.nn.functional as functional
import torch.autograd as autograd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable

In [6]:
import torchvision.datasets as dsets

In [7]:
import torchvision.transforms as transforms

In [17]:
import torch.nn as nn

In [16]:
mb_size = 64
Z_dim = 100
X_dim = 784
y_dim = 10
h_dim = 128
c = 0
lr = 1e-3

In [8]:
train_data = dsets.MNIST(root="./datasets/", train=True,
                        transform=transforms.ToTensor(),
                        download=True)

In [10]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                          shuffle=True, drop_last=True,
                                           batch_size=mb_size)

In [11]:
all_data = {}
for i, data in enumerate(train_loader):
    all_data[i] = data

In [40]:
# def Q(z|X)

class Q_(nn.Module):
    def __init__(self):
        super(Q_, self).__init__()
        self.fc1 = nn.Linear(X_dim, h_dim)
        self.z_mu = nn.Linear(h_dim, Z_dim)
        self.z_var = nn.Linear(h_dim, Z_dim)
        self.relu = nn.ReLU()
        
    def forward(self,X):
        h = self.relu(self.fc1(X))
        zmu = self.z_mu(h)
        zvar = self.z_var(h)
        return zmu, zvar

In [41]:
def sample_z(mu, log_var):
    eps = Variable(torch.randn(mb_size, Z_dim))
    return mu + torch.exp(log_var / 2) * eps

In [42]:
def one_hot_vector(label, num_classes):
    eye = torch.eye(num_classes)
    return eye[label]

In [43]:
q = Q_()

In [27]:
solver = torch.optim.Adam(q.parameters(), lr=lr)

In [28]:
data = all_data.get(1)

In [29]:
img, label = data

In [30]:
img.size(), label.size()

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [31]:
img = img.view(mb_size, -1)

In [37]:
label = one_hot_vector(label, 10)

In [38]:
label.size()

torch.Size([64, 10])

In [45]:
img = Variable(img); label = Variable(label)

In [46]:
blmu, blvar = q(img)